# PREN2
Path to recognize these little icons
## Data Preparation for the training
First we need to prepare image data and icons, therefore they must fit to the desired training
### Download Targetpictograms
Convert and crop them with *imagemagick*
```console
convert -crop 1000x1000-10-50 -gravity center +repage -density 150 Piktogramme_PREN1_HS20.pdf -quality 100 -resize 200x200 output-img.jpg
```

## Negative Samples
Negative Samples sind beliebige Bilder, welche das Objekt nicht enthalten dürfen.
1. Download Negative Bilder https://github.com/sonots/tutorial-haartraining/tree/master/data/negatives
1. in Arbeitsverzeichnis ablegen
1. TXT-Dokument mit Pfad zu allen negativen Bilder erstellen: `find negatives -type f > negatives.txt`
1. Da create_samples-Tool in der Schlaufe immer die gleichen Bilder verwendete negateives negatives.txt aufsplitten: `split -l 230 negatives.txt`
1. Filenames anpassen, für jedes Piktogramm ein eigenes File

## Positive Samples
1. Verzeichnisse für jedes Piktogramm anlegen
1. Mit Script Command erzeugen lassen
1. Für jedes Piktogramm Befehl absetzen `/Users/stofers/openCV/opencv/cmake-build-debug/bin/opencv_createsamples -info /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer.txt -bg /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/negatives_hammer.txt -img /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/sources/hammer.jpg -num 1500 -w 24 -h 24 -maxxangle 0.2 -maxzangle 0.2 -maxyangle 0.2 -bgcolor 100`
1. Vec Datei erzeugen: `/Users/stofers/openCV/opencv/cmake-build-debug/bin/opencv_createsamples -info /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer.txt -bg /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/negatives_hammer.txt -vec /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training/hammer/hammer.vec -num 1500 -w 24 -h 24`
1. Verzeichnis Training zippen
1. Server schicken `scp /Users/stofers/Development/HSLU/PREN/pren_stair_climb/Raspberry_Pi/pictograms/training.zip localadm@prenh20-sstofer.el.eee.intern:~`
1. Auf server entpacken `unzip training.zip`
1. opencv_trainingcascade must be in same directory as the data!
1. run training with `./opencv_traincascade -data hammer -vec hammer/hammer.vec -bg negatives_hammer.txt -numPos 1500 -numNeg 1500 -numStages 15 -w 24 -h 24`
1. Copy cascade.xml to local directory `scp localadm@prenh20-sstofer.el.eee.intern:~/training/hammer/cascade.xml hammer/cascade.xml`

## Verification

In [ ]:
import cv2

classifier = "hammer"
item_clsfr=cv2.CascadeClassifier("../"+classifier+"/cascade.xml")
#loading the cascade classifier

camera=cv2.VideoCapture(0)
#initializing the video object (0 for default webcam)

while(True):
#infinite loop to read continuous frames from the camera object

    ret,img=camera.read()
    #reading a single frame from the camera
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #converting the color image to a gray scale image
    items=item_clsfr.detectMultiScale(gray)     
    #detecting in the gray scale
    #hammers is a 2D array contaning n number of rows (n= number of hammers in the frame), 4 columns (x,y,w,h)
    for (x,y,w,h) in items:
    #going through each and assigning the x,y,w,h

        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        #Drawing a rectangle bounding the faces
        cv2.putText(img,classifier,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            
    cv2.imshow('LIVE',img)
    cv2.waitKey(1)
    #showing the frame